In [26]:
import os 
from dotenv import load_dotenv
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
load_dotenv()

True

In [23]:
from openai import OpenAI
import httpx
import os

# As mentioned in the above Note try to setup Dell certificate in your environment to avoid verify=False(SSL verification is disabled).
http_client=httpx.Client(verify=False)
client = OpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    http_client=http_client,
    api_key=os.environ['LLM_API']
)

In [3]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.read()
len(required_prompt)

56548

In [24]:
streaming = True
max_output_tokens = 100

# Read the content of 'output.txt' as the required prompt
# with open('../output.txt', 'r') as f:
#     required_prompt = f.read()

# Available Models list
available_models = ["mixtral-8x7b-instruct-v01", "llamaguard-7b", "gemma-7b-it", "mistral-7b-instruct-v02", "phi-2", "llama-2-70b-chat", "phi-3-mini-128k-instruct", "llama-3-8b-instruct"]

# Let's select the model from the available list
model_selected = available_models[0]


print(f"Model: {model_selected}")
completion = client.completions.create(
    model=model_selected,
    max_tokens=max_output_tokens,
    prompt="Hi, I am bob, tell me about yourself.",
    stream=streaming
)

if streaming:
    for chunk in completion:
        print(chunk.choices[0].text, end='')
else:
    print(completion.choices[0].text)


Model: mixtral-8x7b-instruct-v01

Hi Bob, I am a bot, I analyze input and help find ways to solve problems. How can I help you today?

Recently, there's been a seemingly endless number of generative AI projects, from "fine-tuned" public models to fully proprietary closed systems. These models help us do everything from drafting emails to creating images from textual descriptions, but they often lack the ability to control the tone or style of output.

I

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 7f4a3b7a-c43e-4d4d-8d02-32a536c98eae not found for run b5e7127d-d482-4809-af11-4fa954e8ebde. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. How can I help you today? Is there something you would like to know or talk about? I'm here to answer any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 14, 'total_tokens': 62}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5e7127d-d482-4809-af11-4fa954e8ebde-0', usage_metadata={'input_tokens': 14, 'output_tokens': 48, 'total_tokens': 62})

In [14]:



store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
        return store[session_id]

    memory = ConversationBufferWindowMemory(
        chat_memory=store[session_id],
        k=3,
        return_messages=True,
    )
    assert len(memory.memory_variables) == 1
    key = memory.memory_variables[0]
    messages = memory.load_memory_variables({})[key]
    store[session_id] = InMemoryChatMessageHistory(messages=messages)
    return store[session_id]

llm = ChatOpenAI(base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client)

chain = RunnableWithMessageHistory(llm, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}},
)  # session_id determines thread

Parent run 01dee4e9-428e-4a98-9585-bec7cf7ef13b not found for run fb4faafb-4435-4391-aa85-d8c13d9b8e3b. Treating as a root run.


AIMessage(content=" Hello Bob! It's nice to meet you. Is there something specific you would like to talk about or ask me? I'm here to help with any questions you have to the best of my ability.", response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59}, 'model_name': 'mixtral-8x7b-instruct-v01', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fb4faafb-4435-4391-aa85-d8c13d9b8e3b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59})

In [15]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    "Hi, I'm Bob. What's your name?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Nice to meet you! What do you know about Python?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")

# Third interaction with a different session
print("\nThird interaction (different session):")
response3 = chain.invoke(
    "Hello, I'm Alice. Can you tell me about the weather?",
    config={"configurable": {"session_id": "2"}},
)
print("Assistant:", response3.content)

# Check history for both sessions
print_chat_history("1")
print_chat_history("2")

Model: mixtral-8x7b-instruct-v01


Parent run c7cca3da-4cb6-472b-9339-f6b837bd0f2d not found for run 8ce21e30-5c35-4f4e-8950-b32f3cddf423. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 5207cc8d-87b3-4ca8-aa61-2aff9e8de81c not found for run 64872ab4-ee3d-409e-8be6-b81d69112391. Treating as a root run.


Assistant:  Hello Bob, I'm an assistant. I don't have a personal name as I'm here to help you with any questions or information you need. How can I assist you today?

Chat History for session 1:
human: Hi, I'm Bob. What's your name?
ai:  Hello Bob, I'm an assistant. I don't have a personal name as I'm here to help you with any questions or information you need. How can I assist you today?

Second interaction:


Parent run a537d0d0-8aa3-4b3b-b6e7-c0355b3f4b62 not found for run f6216de6-c11b-42bb-8a9b-026fca68710d. Treating as a root run.


Assistant:  Hello Bob, nice to meet you too! I can tell you that Python is a high-level, interpreted programming language that was created by Guido van Rossum and first released in 1991. It is designed with an emphasis on code readability, and its syntax allows programmers to express concepts in fewer lines of code than would be possible in languages such as C++ or Java.

Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming. It has a large standard library that is available for use without additional installation, which includes modules for working with files, interacting with operating systems, and handling common data structures.

Python is widely used in various domains, such as web development, scientific computing, data analysis, artificial intelligence, machine learning, and automation. It also has a large and active community that contributes to its development and provides support to users.

Chat History for session 1

In [16]:
with open('../output1.txt', 'r') as f:
    required_prompt = f.readlines()

In [17]:
type(required_prompt)

list

In [18]:
required_prompt[:10]

["vram\\EmulatorPkg\\Unix\\lldbinit:imports-['command script import lldbefi.py']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\configure-cmake:imports-[\'#   You must include the GNUInstallDirs module (which comes with\', \'#   CMake) in your project.  Just put "include (GNUInstallDirs)" in\']\n',
 "vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\Makefile:imports-['$(CC) $(CFLAGS) $(CPPFLAGS) -Ic/include \\\\']\n",
 'vram\\MdeModulePkg\\Library\\BrotliCustomDecompressLib\\brotli\\python\\Makefile:imports-[\'EXT_SUFFIX=$(shell $(PYTHON) -c \\\'import sysconfig; print(sysconfig.get_config_var("EXT_SUFFIX"))\\\')\']\n',
 'vram\\MdeModulePkg\\Universal\\RegularExpressionDxe\\oniguruma\\INSTALL:imports-[\'below.  The lack of an optional feature in a given package is not\', \'The simplest way to compile this package is:\', "\'./configure\' to configure the package for your system.", \'recommended that the package be configured and built as a regular\',

In [19]:
required_prompt = "".join(required_prompt[:30])

In [20]:
print(len(required_prompt))

8514


Feed this into a conversational chain

In [27]:
print(f"Model: {model_selected}")

# Chat history management
store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

def print_chat_history(session_id: str):
    history = get_session_history(session_id)
    print(f"\nChat History for session {session_id}:")
    for message in history.messages:
        print(f"{message.type}: {message.content}")

# Create a proper LLM object
llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)

# Check initial state
print_chat_history("1")

# First interaction
print("\nFirst interaction:")
response1 = chain.invoke(
    required_prompt+"Understand the codebase and the project structure.",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response1.content)

# Check history after first interaction
print_chat_history("1")

# Second interaction
print("\nSecond interaction:")
response2 = chain.invoke(
    "Can you generate test automation scripts on this project given the information or do you need more?",
    config={"configurable": {"session_id": "1"}},
)
print("Assistant:", response2.content)

# Check history after second interaction
print_chat_history("1")



Model: mixtral-8x7b-instruct-v01


Parent run e1666d88-233b-492c-8be1-b742ab26a60d not found for run 5e888432-2367-470f-b70e-e89906f26dc0. Treating as a root run.



Chat History for session 1:

First interaction:


Parent run 04d9e985-0978-4fa0-8c85-937bcadd5eca not found for run 1abbd9e5-6275-4201-99c0-2485c9d0e699. Treating as a root run.


Assistant:  The text you provided appears to be fragments of various configuration and documentation files from a codebase. I will attempt to summarize the information contained in these fragments.

1. lldbinit: This file is related to the LLDB debugger and imports a Python module `lldbefi.py`.
2. configure-cmake: This file is a CMake configuration script for the BrotliCustomDecompressLib library and mentions the GNUInstallDirs module.
3. Makefile: This is a Makefile for the BrotliCustomDecompressLib library, using the C compiler and including headers from the `c/include` directory.
4. python/Makefile: This is a Makefile for building Python extensions in the BrotliCustomDecompressLib library, extracting the extension suffix from the Python configuration.
5. INSTALL: This is a generic installation instruction file for the oniguruma package, mentioning various ways to configure and compile the package for different systems and environments.
6. depcomp: This file is an Autoconf script for

It kinda works I think the prompts need to be more streamlined and then I can work on the post processing and preprocessing of the file

In [30]:
with open('../output1.txt', 'r') as f:
    content = f.readlines()

store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

llm = ChatOpenAI(
    base_url='https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1',
    model=model_selected,
    openai_api_key=os.environ['LLM_API'],
    http_client=http_client
)

chain = RunnableWithMessageHistory(llm, get_session_history)


current_context = "".join(content[:10])
ctr = 10
llm_request = "need more"
while llm_request == "need more":
    print(f"Current Context: {current_context}")
    response = chain.invoke(
        current_context+" generate test automation scripts for this part of the code or request for more information and you can only say 'need more' to request for more information.",
        config={"configurable": {"session_id": "1"}},
    )
    print("Assistant:", response.content)
    if 'need' in response.content.lower() or 'more' in response.content.lower():
        llm_request = "need more"
        ctr+=10
        current_context = "".join(content[ctr:ctr+10]) if ctr+10 < len(content) else "".join(content[ctr:])
    else:
        llm_request = "done"



Parent run a3c438e5-f2df-46fe-ac35-cea33a387293 not found for run 9fcdc17b-c201-4b41-855b-e4fd8aa157a8. Treating as a root run.


Current Context: vram\EmulatorPkg\Unix\lldbinit:imports-['command script import lldbefi.py']
vram\MdeModulePkg\Library\BrotliCustomDecompressLib\brotli\configure-cmake:imports-['#   You must include the GNUInstallDirs module (which comes with', '#   CMake) in your project.  Just put "include (GNUInstallDirs)" in']
vram\MdeModulePkg\Library\BrotliCustomDecompressLib\brotli\Makefile:imports-['$(CC) $(CFLAGS) $(CPPFLAGS) -Ic/include \\']
vram\MdeModulePkg\Library\BrotliCustomDecompressLib\brotli\python\Makefile:imports-['EXT_SUFFIX=$(shell $(PYTHON) -c \'import sysconfig; print(sysconfig.get_config_var("EXT_SUFFIX"))\')']
vram\MdeModulePkg\Universal\RegularExpressionDxe\oniguruma\INSTALL:imports-['below.  The lack of an optional feature in a given package is not', 'The simplest way to compile this package is:', "'./configure' to configure the package for your system.", 'recommended that the package be configured and built as a regular', "files that 'configure' created (so you can compile 

Parent run bc3dd4d2-90e1-4e72-9286-43bfa22d67e8 not found for run 19751b06-e2f5-4083-afa5-5d0b7c13b409. Treating as a root run.


Assistant:  It seems like you have provided me with various code snippets and descriptions related to different software packages. However, you have not asked a specific question or given context about what you would like me to help you with.

Based on the code snippets you have provided, it seems like they are related to different software packages such as `lldb`, `BrotliCustomDecompressLib`, `RegularExpressionDxe`, and `oniguruma`.

If you have a specific question related to any of these packages or need help with something specific, please provide more context and clarify your question. Otherwise, I can provide some general information about these packages:

* `lldb` is a next generation, high-performance debugger. It is built as a set of reusable components which highly leverage existing libraries in the larger LLVM Project.
* `BrotliCustomDecompressLib` is a custom decompression library for the Brotli compression algorithm.
* `RegularExpressionDxe` is a regular expression library 

Parent run 1771e06a-b6d3-4a72-96e6-0640af4d801a not found for run a76899f8-df5b-46f9-8061-fd9cbae4dd57. Treating as a root run.


...

Parent run e66768ed-24c2-4349-9fea-3c29d7f2c5c7 not found for run 2e27fbf7-0587-46be-bc69-d4a5b952a8e9. Treating as a root run.


Assistant:  The code snippets you have provided are from various VFR (Variable Field Reference) files for different modules and drivers in the EDK II project. Here's a brief description of each file:

* `DriverHealthConfigureVfr.Vfr`: This file is from the `DriverHealthManagerDxe` module and contains HII data for the driver health manager configuration.
* `Sample.vfr`: This file is from the `HiiResourcesSampleDxe` module and contains HII data for a sample HII driver.
* `Vfr.vfr`: This file is from the `PlatformDriOverrideDxe` module and contains HII data for the platform driver override.
* `HttpBootConfigVfr.vfr`: This file is from the `HttpBootDxe` module and contains HII data for the HTTP boot configuration.
* `IScsiConfigVfr.vfr`: This file is from the `IScsiDxe` module and contains HII data for the iSCSI configuration.
* `PlatformForms.vfr`: This file is from the `PlatformDxe` module and contains HII data for the platform configuration.
* `TlsAuthConfigVfr.vfr`: This file is from t

Parent run 6d87cce7-5815-486d-af1c-a1b9c290c3bc not found for run f2b208b0-e56f-44cb-8c4e-9b63fc8dee09. Treating as a root run.


Assistant:  The code snippets you have provided are from various VFR (Variable Field Reference) files and other files related to different modules and drivers in the EDK II project. Here's a brief description of each file:

* `Tcg2Config.vfr`: This file is from the `Tcg` module and contains HII data for the TCG2 configuration.
* `OpalPasswordForm.vfr`: This file is from the `Opal` module and contains HII data for the Opal password configuration.
* `API.ja`: This file is from the `oniguruma` module and contains Japanese documentation for the oniguruma regular expression library.
* `CALLOUTS.API.ja`: This file is from the `oniguruma` module and contains Japanese documentation for the oniguruma regular expression library callout functions.
* `SecureBootConfig.vfr`: This file is from the `SecureBootConfigDxe` module and contains HII data for the secure boot configuration.
* `API.ja`: This file is from the `oniguruma` module and contains Japanese documentation for the oniguruma regular expr

BadRequestError: Error code: 400 - {'object': 'error', 'message': 'max_tokens must be at least 1, got -116807.', 'type': 'invalid_request_error', 'param': None, 'code': None}